In [1]:
import pandas as pd
import numpy as np
import openpyxl as ox
import re
import editdistance
from datetime import datetime

In [2]:
# надо ли переделать на создание без извлечения из иксель?
df = pd.DataFrame(columns=['Менеджер', 'Вид доставки', 'Дата', 'Товар', 'Примечание к Товару', 
                           'Кол-во', 'Ед.изм.', 'Машина/Водитель','Продавец', 'Откуда', 
                           'Покупатель', 'Грузополучатель ', 'Адрес грузополучателя (юрид)', 
                           'Адрес пункта разгрузки', 'Контакт гп','Примечание Иное',
                          ])


In [3]:
df

,Менеджер,Вид доставки,Дата,Товар,Примечание к Товару,Кол-во,Ед.изм.,Машина/Водитель,Продавец,Откуда,Покупатель,Грузополучатель,Адрес грузополучателя (юрид),Адрес пункта разгрузки,Контакт гп,Примечание Иное


In [4]:
managers = pd.read_excel("dictionary.xlsx", sheet_name="managers")
products = pd.read_excel("dictionary.xlsx", sheet_name="products")
units = pd.read_excel("dictionary.xlsx", sheet_name="units")

In [125]:
# managers = pd.DataFrame({'login': ['ИГО', 'Юра Менеджер', 'Алексей Мельхер'],
#                         'manager': ['Игорь Хабаров', 'Юрий', 'Алексей Мельхер']})

In [126]:
# products = pd.Series(data=['ЦЕМ I 42.5Н БЦК','Щебень гранитный 5-20 (ЛСР)','ЦЕМ I 42.5Н Костюковичи',
#                                'ЦЕМ I 42.5Н БЦЗ Костюковичи', 'ЦЕМ I 42.5Н', 'ПЕСКОГРУНТ'], name='product')

In [104]:
with pd.ExcelWriter("dictionary.xlsx") as writer:
    managers.to_excel(writer, sheet_name="managers", index=False)
    products.to_excel(writer, sheet_name="products", index=False)

In [57]:
products.str.replace(' ', '')

0                ЦЕМI42.5НБЦК
1    Щебеньгранитный5-20(ЛСР)
2        ЦЕМI42.5НКостюковичи
3     ЦЕМI42.5НБЦЗКостюковичи
4                   ЦЕМI42.5Н
5                  ПЕСКОГРУНТ
dtype: object

## Text

In [287]:
appl = """
Юра Менеджер:
Заявка на самовывоз
1. Дата отгрузки: 27.04.24 
2. Марка: ПЕСКОГРУНТ
Обязательно актуальный паспорт!
3. Количество: 200 кубов 
4. Продажа от клиента: ООО Спарта
5. Завод отгрузки: КПП, Солнечногорск, Люберцы 
6. Покупатель: ООО "КАНТАН"
7. Грузополучатель: АО "МСУ-1"
8. Время приёмки: с 9.00 до 20.00
9. Контакт диспетчера: +7 968 558-83-13 
10. Точка выгрузки: Бескудниковский бульвар, 52к1
Россия, Москва
7. Машина: 
О327ВН790 МАN
О039ОУ790 МАN,
Х194ВА797 КАМАЗ
В683СН790 МАN
"""

In [198]:
# Используем регулярное выражение для поиска даты в формате dd.mm.yyyy
date_pattern = r'\b\d{2}\.\d{2}(?:\.\d{2}(?:\d{2})?)?\b'

# Находим все совпадения с помощью метода findall
dates = re.findall(date_pattern, appl)

## Check

In [199]:
dates

['27.04.24', '20.00']

In [200]:
# функция для форматирования даты в установленный формат
def convert_to_full_year(date_str):
    current_year = datetime.now().year
    parts = date_str.split('.')
    
    # Пытаемся интерпретировать дату
    try:
        # Если в строке только две части, добавляем текущий год
        if len(parts) == 2:
            date_str += f".{current_year}"
        
        # Пробуем преобразовать строку в дату
        datetime_obj = datetime.strptime(date_str, '%d.%m.%Y')
    except ValueError:
        # Если возникает ошибка, попробуем считать год в формате yy
        try:
            datetime_obj = datetime.strptime(date_str, '%d.%m.%y')
        except ValueError:
            # Если и это не сработает, возможно, формат даты неизвестен
            raise ValueError("Неподдерживаемый формат даты")
    
    # Возвращаем дату в формате dd.mm.yyyy
    return datetime_obj.strftime('%d.%m.%Y')

In [201]:
# конвертируем ПОКА ЧТО для одной даты
convert_date = convert_to_full_year(dates[0])

## Check

In [202]:
convert_date

'27.04.2024'

In [203]:
def find_manager(text, managers_df):
    # Проходим по каждому логину менеджера
    for index, row in managers_df.iterrows():
        login = row['login']
        manager = row['manager']
        
        # Проверяем совпадение по колонке 'login'
        if login.lower().replace(' ', '') in text.lower().replace(' ', ''):
            return manager
    return np.NAN



In [204]:
manager_found = find_manager(appl, managers)

In [205]:
manager_found

'Юрий'

In [256]:
def delivery_type(text):
    if "самовывоз" in text.lower():
        return "самовывоз"
    return "доставка"

In [257]:
delivery_found = delivery_type(appl)

In [258]:
delivery_found

'самовывоз'

In [209]:
# def find_product(text, product_df):
#     """
#     Функция для поиска товара по тексту и датафрейму с видами товара.

#     Аргументы:
#     text (str): Текст для поиска товара.
#     product_df (pandas.DataFrame): Датафрейм с колонками 'login' и 'manager',
#                                      содержащий информацию о менеджерах.

#     Возвращает:
#     str: Имя менеджера, если совпадение найдено, иначе сообщение о том, что совпадений не найдено.
#     """
#     # Проходим по каждому логину менеджера
#     for index, row in product_df.iterrows():
#         product = row['product']
#         # Проверяем совпадение по колонке 'login'
#         if product.lower().replace(' ', '') in text.lower().replace(' ', ''):
#             return product
#     # Если совпадений не найдено
#     return np.NAN

In [210]:
# product_found = find_product(appl, products)

In [211]:
# product_found

In [275]:
def find_product(text):
    # Находим значение после слова "Марка:" без учета регистра
    marca_match = re.search(r'(?i)Марка(:)?\s*(цемента)?\s*(.*?)\n', text)
    marca_value = marca_match.group(3) if marca_match else np.NAN

    return marca_value
    
def find_product_notice(text):    
    # Находим значение между строкой с "Марка:" и следующим номером пункта
    # Если нет следующего номера пункта, возвращаем np.NAN
  
    intermediate_match = re.search(r'(?i)Марка.*?\n(.*?)\n\d', text, re.DOTALL)
    intermediate_value = intermediate_match.group(1) if intermediate_match else np.NAN
    
    if re.match(r'^[^0-9.].*', intermediate_value):
        return intermediate_value
    else:
        np.NAN

In [276]:
product_found = find_product(appl)

In [277]:
product_notice_found = find_product_notice(appl)

## Check

In [278]:
product_found

'ПЕСКОГРУНТ'

In [279]:
product_notice_found

'Обязательно актуальный паспорт!'

In [271]:
def find_quantity(text):
    # Находим значение после слова "Количество/кол-во:" без учета регистра    
    quantity_match = re.search(r'(?i)кол(\s*-\s*|ичест)?во\s*(:)?\s*(\d+)', text)
    quantity = int(quantity_match.group(3)) if quantity_match else np.NAN
    
    return quantity

In [272]:
quantity_found = find_quantity(appl)

In [273]:
quantity_found

200

In [219]:
def find_unit_note(text):
    # Находим значение после слова "Количество/кол-во:" без учета регистра    
    unit_match = re.search(r'(?i)кол(-|ичест)?во\s*(:)?\s*(\d+)\s*(\D+)\s*', text)
    unit = unit_match.group(4).strip() if unit_match else np.NAN
    
    return unit

In [220]:
unit_note_found = find_unit_note(appl)

In [221]:
unit_found = units.loc[units[units['data']==unit_note_found].index[0], 'unit']

In [226]:
def find_car(text):
    
    car_match = re.findall(r'(?i)[А-Я]{1}\d{3}[А-Я]{2}\d{3}\s*\w*', text)
    cars = car_match if car_match else np.NAN
    
    return cars

In [227]:
cars_found = ', '.join(find_car(appl)) if find_car(appl) is not np.NAN else np.NAN

In [228]:
cars_found

'О327ВН790 МАN, О039ОУ790 МАN, Х194ВА797 КАМАЗ, В683СН790 МАN'

In [262]:
def find_organization(text):
#     Продажа от: ООО "Спарта"
    organization_match = re.search(r'(?i)(продажа\s+от:|(продажа)?\s*от\s+(клиента)?\s*(:)?)\s*(.+?)\n', text)
    organization = organization_match.group(5) if organization_match else np.NAN
    
    return organization

In [263]:
organization_found = find_organization(appl)

In [264]:
organization_found

'ООО Спарта'

In [266]:
def find_transshipment(text):
    
    transshipment_match = \
    re.search(r'(?i)\d+\.\s*(С\s+(перевалки)?\s*|Завод\s*(отгрузки)?\s*(:)?|Перевалка\s*(:)?)\s*(.+?)\n', 
              text)
    transshipment = transshipment_match.group(6) if transshipment_match else np.NAN
    
    return transshipment

In [267]:
transshipment_found = find_transshipment(appl)

In [268]:
transshipment_found 

'КПП, Солнечногорск, Люберцы '

## Доделать, спросить у Лёхи

Нужно ли создавать словарь для именительного падежа названия перевалки?

In [288]:
def find_purchaser(text):
    
    purchaser_match = \
    re.search(r'(?i)\d+\.\s*(Покупатель\s*(груза)?\s*(:)?)\s*(.+?)\n', 
              text)
    purchaser = purchaser_match.group(4) if purchaser_match else np.NAN
    
    return purchaser

In [290]:
purchaser_found = find_purchaser(appl)

  ## Доделать, спросить у Лёхи

4. ООО «ПД Технологии» объект *Пушкино* ,
4. ООО «ПД Технологии» объект *Пушкино* 

In [293]:
# Создаем новый DataFrame с одной строкой, содержащей только дату
new_row_df = pd.DataFrame({'Менеджер': [manager_found], 
                           'Вид доставки': [delivery_found], 
                           'Дата': [convert_date], 
                           'Товар': [product_found], 
                           'Примечание к Товару': [product_notice_found], 
                           'Кол-во': [quantity_found], 
                           'Ед.изм.': [unit_found], 
                           'Машина/Водитель': [cars_found], 
                           'Продавец': [organization_found], 
                           'Откуда': [transshipment_found], 
                           'Покупатель': [purchaser_found], 
#                            'Грузополучатель ',
#                            'Адрес грузополучателя (юрид)', 
#                            'Адрес пункта разгрузки', 
#                            'Контакт гп',
#                            'Примечание Иное',
                          })

# new_row_df['Машина/Водитель'] = new_row_df['Машина/Водитель'].apply(lambda x: ', '.join(x)) 
# Объединяем исходный DataFrame с новым DataFrame
df = pd.concat([df, new_row_df], ignore_index=True)

In [294]:
df

,Менеджер,Вид доставки,Дата,Товар,Примечание к Товару,Кол-во,Ед.изм.,Машина/Водитель,Продавец,Откуда,Покупатель,Грузополучатель,Адрес грузополучателя (юрид),Адрес пункта разгрузки,Контакт гп,Примечание Иное
0,Юрий,доставка,26.04.2024,Щебень гранитный 5-20(ЛСР),Обязательно актуальный паспорт!,120,т,"В683СН790 МАN, М991ХС750 МАN, О030ХЕ123 МАN, В...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Юрий,доставка,26.04.2024,Щебень гранитный 5-20(ЛСР),Обязательно актуальный паспорт!,120,т,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Юрий,доставка,26.04.2024,Щебень гранитный 5-20(ЛСР),Обязательно актуальный паспорт!,120,т,NaN,"ОО0 ""Спарта""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Юрий,доставка,26.04.2024,Щебень гранитный 5-20(ЛСР),Обязательно актуальный паспорт!,120,т,NaN,"ОО0 ""Спарта""",🤷‍♂️,NaN,NaN,NaN,NaN,NaN,NaN
4,Юрий,доставка,27.04.2024,ПЕСКОГРУНТ,Обязательно актуальный паспорт!,200,куб,"О327ВН790 МАN, О039ОУ790 МАN, Х194ВА797 КАМАЗ,...","ООО ""Спарта""",🤷‍♂️,NaN,NaN,NaN,NaN,NaN,NaN
5,Юрий,самовывоз,27.04.2024,ПЕСКОГРУНТ,Обязательно актуальный паспорт!,200,куб,"О327ВН790 МАN, О039ОУ790 МАN, Х194ВА797 КАМАЗ,...","ООО ""Спарта""",🤷‍♂️,NaN,NaN,NaN,NaN,NaN,NaN
6,Юрий,самовывоз,27.04.2024,ПЕСКОГРУНТ,Обязательно актуальный паспорт!,200,куб,"О327ВН790 МАN, О039ОУ790 МАN, Х194ВА797 КАМАЗ,...",ООО Спарта,"КПП, Солнечногорск, Люберцы",NaN,NaN,NaN,NaN,NaN,NaN
7,Юрий,самовывоз,27.04.2024,ПЕСКОГРУНТ,Обязательно актуальный паспорт!,200,куб,"О327ВН790 МАN, О039ОУ790 МАN, Х194ВА797 КАМАЗ,...",ООО Спарта,"КПП, Солнечногорск, Люберцы",NaN,NaN,NaN,NaN,NaN,NaN
8,Юрий,самовывоз,27.04.2024,ПЕСКОГРУНТ,Обязательно актуальный паспорт!,200,куб,"О327ВН790 МАN, О039ОУ790 МАN, Х194ВА797 КАМАЗ,...",ООО Спарта,"КПП, Солнечногорск, Люберцы","ООО ""КАНТАН""",NaN,NaN,NaN,NaN,NaN


In [75]:
df.drop(df.index, inplace=True)

In [76]:
df.columns

Index(['Менеджер', 'Вид доставки', 'Дата', 'Товар', 'Примечание к Товару',
       'Кол-во', 'Ед.изм.', 'Машина/Водитель', 'Продавец', 'Откуда',
       'Покупатель', 'Грузополучатель ', 'Адрес грузополучателя (юрид)',
       'Адрес пункта разгрузки', 'Контакт гп', 'Примечание Иное'],
      dtype='object')

In [ ]:
patterns = [
    # ПАО ГТС => ГТС, ПАО
    r'^(?i)(П?АО|ООО|ГУП)(?:\s+|\s*")([^"]+)"?',
    
    # ПАО Р-БАНК//ИЗМАЙЛОВ ВАЛЕРИЙ МИХАЙЛОВИЧ//РОССИЯ,МОСКВА Г Р/С в ПАО Р-БАНК г Москва => Измайлов В.
    r'//([\w-]+)\s+(\w)\w*\s+(?:\w)\w*//',
    
    # Индивидуальный ПРЕДПРИНИМАТЕЛЬ Логинов Андрей Николаевич => Логинов А., ИП
    r'^(?i)(и)(?:ндивидуальный)?\s*(п)(?:редприниматель)?\s*(\w+)\s+(\w)\w*\s*(\w)?\w*',
    
    # Никишкина Лидия Алексеевна (ИП) => Нокошина Л., ИП
    r'^([\w-]+)\s+(\w)\w*\s*(\w)?\w*\s*\((ИП)\)$',
    
    # ВОРОНОВ ИЛЬЯ АНДРЕЕВИЧ => Воронов И.
    r'([\w-]+)\s+(\w)\w*\s+(\w)\w*$',
    
    # ООО "ЧОП "ОХРАННЫЕ СИСТЕМЫ" => ОХРАННЫЕ СИСТЕМЫ, ЧОП, ООО
    r'^([\w-]{2,3})\s+"?(.*)\s+"(.*)"',
    
    #  contains "ФНС": 
    r'\w*\s*(?i)(?:и?)\s*(ф)(?:едеральной)?\s*(н)(?:алоговой)?\s*(с)(?:лужбы)?\s*(?:россии)?\s*\w*\s*', 
    
    # ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЛАГ1" => ЛАГ1, ООО
    r'(\w)\w{3,}|\"([^\"]+)\"',
    
    # Межрегиональное операционное УФК (КАЗНАЧЕЙСТВО РОССИИ) => Межрегиональное операционное УФК
    r'^.+?(?=\s*\()',
]       

def extract_text(row):
    """
    Get string with raw title, extract necessary text by patterns and returns formatted values
    """
    matches = []

    for pattern in patterns:
        match = re.findall(pattern, row)
        matches.append(match)
    
    return '{}, {}, {}'.format(matches[5][0][2], matches[5][0][1], matches[5][0][0]) if matches[5] else \
        '{} {}.'.format(matches[1][0][0].capitalize(), matches[1][0][1]) if matches[1] else \
        '{}, {}'.format(matches[0][0][1], matches[0][0][0]) if matches[0] else \
        '{} {}.{}., {}{}'.format(matches[2][0][2].capitalize(), matches[2][0][3], matches[2][0][4], matches[2][0][0], matches[2][0][1]) if matches[2] else \
        '{} {}.{}., {}'.format(matches[3][0][0].capitalize(), matches[3][0][1], matches[3][0][2], matches[3][0][3]) if matches[3] else \
        '{}{}{}'.format(matches[6][0][0].upper(),matches[6][0][1].upper(),matches[6][0][2]).upper() if matches[6] else \
        '{} {}.{}.'.format(matches[4][0][0].capitalize(), matches[4][0][1], matches[4][0][2]) if matches[4] else \
        '{}, {}'.format([mat[1] for mat in matches[7] if mat[1]][0],
                        ''.join(map(str, [mat[0] for mat in matches[7] if
                                          mat[0]]))) if '"' in row else \
        '{}'.format(matches[8][0]) if matches[8] else \
        row

# creating column "Agent"
df['Agent'] = np.vectorize(extract_text)(df['Full_Name'])
df['Name'] = df['Agent'].map(lambda x: x.lower())

In [95]:
def fix_typo(original, target):
    distance = editdistance.eval(original, target)
    if distance <= 3:  # Пороговое значение для определения опечатки
        return target
    else:
        return original

original = "Щебин ранитный"
target = "Щебень гранитный"
fixed_string = fix_typo(original, target)
print("Fixed string:", fixed_string)

Fixed string: Щебень гранитный


In [139]:
import re
import numpy as np

text = """
1. Дата отгрузки: 28.04.24 
2. Марка Цем I 42.5Н Кричев
3. Количество: 40т.
"""

# Находим значение после слова "Марка:" без учета регистра
marca_match = re.search(r'Марка\s*(цемента)?\s*(.*?)\n', text, re.IGNORECASE)
marca_value = marca_match.group(2)

# Находим значение между строкой с "Марка:" и следующим номером пункта
# Если нет следующего номера пункта, возвращаем np.NAN
intermediate_match = re.search(r'Марка.*?\n(.*?)\n\d', text, re.IGNORECASE | re.DOTALL)
intermediate_value = intermediate_match.group(1) if intermediate_match else np.NAN

print("Значение после слова 'Марка:':", marca_value)
print("Значение между строкой с 'Марка:' и следующим номером:", intermediate_value)


Значение после слова 'Марка:': Цем I 42.5Н Кричев
Значение между строкой с 'Марка:' и следующим номером: nan


In [64]:
pattern = r'\d+\..*'
text = "5. Надо полис"

match = re.match( r'\d+\..*', text)

In [65]:
match[0]

'5. Надо полис'

In [83]:
type(re.match(r'^[^0-9.].*', "5. Надо полис"))

NoneType

In [67]:
import re

text="""
2. Марка: ЦЕМ I 42,5н БЦЗ Костюковичи 
Машина с пломбами и актуальными паспортами 
3. Количество: 40 тю.
4. Продажа от: ООО "Спарта"
"""

# Ищем строку, содержащую "Количество:"
match = re.search(r'(?i)кол(-|ичест)?во\s*(:)?\s*(\d+)\s*(\D+)\s*', text)
# Если найдено соответствие, выводим символы после числа в этой строке
if match:
    print(match.group(4))


тю.



In [83]:
import re

def find_unit(text):
    # Находим значение после слова "Количество/кол-во:" без учета регистра    
    unit_match = re.search(r'(?i)кол(-|ичест)?во\s*(:)?\s*\d+\s*(\D+)\s*', text)
    unit = unit_match.group(3) if unit_match else None
    
    return unit

appl = """
Юра Менеджер:
1. Дата отгрузки:
26.04                    
2. От ООО Спарта
3. Марка: Щебень гранитный 5-20(ЛСР)
Обязательно актуальный паспорт!
4. Покупатель ООО НВЛ ГРУП
5. Бетас 
6.  Количество: 120т
7. Машина: 
О327ВН790 МАN
"""

print(find_unit(appl))


т



In [150]:
import re

text="""
4. Покупатель ООО НВЛ ГРУП
5. Бетас 
6.  Количество 120т
7. Машина: 
О327ВН790 МАN
О039ОУ790 МАN,
х194ВА797 КАМАЗ
В683СН790 МАN
К702ОС797 Куряев
"""

car_numbers = re.findall(r'(?i)[А-Я]{1}\d{3}[А-Я]{2}\d{3}\s*\w*', text)

print(car_numbers)


['О327ВН790 МАN', 'О039ОУ790 МАN', 'х194ВА797 КАМАЗ', 'В683СН790 МАN', 'К702ОС797 Куряев']


In [170]:
import pandas as pd

# Создание DataFrame с вашим списком в одной из ячеек
df = pd.DataFrame({'col_name': [['abc', 'def', 'ghi']]})

# Применение метода str.join() к списку в ячейке и замена значения ячейки
df['col_name'] = df['col_name'].apply(lambda x: ', '.join(x))

# Вывод измененного DataFrame
print(df)


        col_name
0  abc, def, ghi


In [171]:
df = pd.DataFrame({'col_name': [['abc', 'def', 'ghi']]})

In [173]:
df

,col_name
0,"[abc, def, ghi]"


In [121]:
import re

text = "5. Покупатель ООО НВЛ ГРУП\n5"

result = re.search(r'\d+\.\s(.+?)\n', text, re.DOTALL)
if result:
    extracted_text = result.group(1)
    print(extracted_text)


Покупатель ООО НВЛ ГРУП
